In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat May  4 16:21:26 2024

@author: julco
"""
########## IMPORTATION DE LIBRAIRIES ##########

import matplotlib.pyplot as plt
import spacy
import string
import time
import json
import numpy as np
import glob
import pandas
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


########## CREATION DE FONCTIONS ##########

# permet de lire un fichier et renvoie une chaine de caractères
def lirefich(chemin):#lire un fichier
    with open(chemin, encoding="utf-8") as r:
        chaine = r.read()
    return chaine

def n_gramme(listee,nb=int(input("donner in chiffre"))):
    nbr_gram=[]
    for elt in listee:
        if len(elt) > 1:
           nbr_gram.append(elt[:nb])
    return nbr_gram

# permet de récupérer les tokens d'un texte à partir du format Doc de spacy et renvoie une liste, calcul de la durée d'exécution également
def tokeniser_spacy(texte):
  debut = time.time()
  liste_tokens = []
  for token in texte:
    liste_tokens.append(token.text)
  print("Récupération des tokens OK")
  fin = time.time()
  durée = fin - debut
  print("La tokenisation avec spacy dure:", durée)
  return liste_tokens

# permet de récupérer les tokens d'un texte en renvoyant une liste
def tokeniser_split(texte):
    debut = time.time()
    texte = texte.split()
    fin = time.time()
    durée = fin - debut
    print("La tokenisation avec la fonction split dure:", durée)
    return texte

# récupère les lemmes à partir du format Doc de spacy
def lemmatiser(texte):
    liste_lemmes=[]
    for mot in texte:
        liste_lemmes.append(mot.lemma_)
    print("Récupération des lemmes OK")
    return liste_lemmes

# récupère les entités à partir du format Doc de spacy
def trouver_entites(texte):
    liste_entites = []
    for w in texte.ents:
        liste_entites.append(w.text)
    print("Récupération des entités nommées OK")
    return liste_entites

### utilisation des fonction récupérer_vocab_manu et trouver_vocab
# permet de récupérer le vocabulaire à partir d'un Doc en utilisant le Pos-Tagging de spacy et renvoie une liste qui sera lue par la fonction trouver_vocab
# et qui séléctionnera les éléments selon leurs Tag
def recuperer_vocab_manu(texte):
    debut = time.time()
    liste_pos_tag = []
    for m in texte:
        liste_pos_tag.append([m.text, m.pos_, m.is_stop])
    vocabulaire = trouver_vocab(liste_pos_tag)
    fin = time.time()
    durée = fin - debut
    print("Trouver le vocabulaire de manière manuelle prend: ", durée)
    return vocabulaire

def trouver_vocab(liste_pos): #genere un vocabulaire en excluant les stop words, punct, nom propre et space
    voc=[]
    for mini_liste in liste_pos:
            if mini_liste[1]!= "PROPN":
                if mini_liste[1] != "SPACE":
                    if mini_liste[1] != "PUNCT":
                        if mini_liste[2]!= True:
                           voc.append(mini_liste[0])
    return voc


def nettoyer_texte(texte):
    exclude_chars = string.punctuation + string.digits
    translation_table = str.maketrans("", "", exclude_chars)
    cleaned_text = texte.translate(translation_table)
    return cleaned_text


### Utilisation des fonction nettoyer_texte, nettoyer_liste et recuperer vocab_auto
# def nettoyer_texte(texte):
#     # Supprimer les signes de ponctuation
#     texte_sans_ponctuation = re.sub(r'[^\w\s]', '', texte)
#     # Supprimer les chiffres
#     #texte_sans_chiffres = re.sub(r'\d', '', texte_sans_ponctuation)
#     # Supprimer les parenthèses (et le contenu entre elles si nécessaire)
#     #texte_nettoye = re.sub(r'\(.*?\)', '', texte_sans_chiffres)
#     return texte_sans_ponctuation

# def nettoyer_liste(liste):
#     liste_propre = [nettoyer_texte(mot) for mot in liste if not mot.startswith('http')]
#     return [element for element in liste_propre if element]

def recuperer_vocab_auto(texte, language):
    debut = time.time()
    liste = tokeniser_spacy(texte)
    stop_words = set(stopwords.words(f"{language}"))
    liste_filtree = [mot for mot in liste if mot.lower() not in stop_words]
    # liste_propre = nettoyer_liste(liste_filtree)
    fin = time.time()
    durée = fin - debut
    print("Trouver le vocabulaire automatiquement prend: ", durée)
    return liste_filtree

# permet d'enregistrer des listes au format JSON
def sauvegarder_json(liste, nom_liste, langue):
    nom_fichier = f"{nom_liste}_spacy_{langue}.json"
    with open(nom_fichier, "w", encoding="utf-8") as json_file:
        json.dump(liste, json_file, ensure_ascii=False, indent=2)
    print(f"Vocabulaire sauvegardé dans {nom_fichier} avec succès !")


########## MAIN ##########

liste_modeles_langue = ["fr_core_news_sm", "en_core_web_sm", "es_core_news_sm"]
liste_langues_nltk = ["french", "english", "spanish"]

#######
i = 1 # permet de changer simultanément la langue de la librairie NLTK selon le modèle de langue spacy choisi, les deux listes du dessus sont rangées dans le même ordre de langues
#######

modele = liste_modeles_langue[i]
langue = modele.split("_")[0]# permet de récupérer la langue dans le modèle spacy afin d'automatiser l'enregistrement de la langue dans le fichier JSON
print(langue)

nlp = spacy.load(modele)# chargement du modèle de langue spacy choisie avec la variable i
nlp.max_length = 2500000# textes longs donc il faut augmenter la capcité de la librairie


# mettre une boucle ici pour faire les 3 modèles simultanément ????????????? il faudrait aussi ajouter un i += 1 à la fin de la boucle por changer de langue nltk
# chemin = "/content/drive/MyDrive/Colab/fr/test/*" # chemin pour Google Colab
chemin = f"corpus_multi/corpus_multi/{langue}/*/*" # chemin pour Spyder

liste_textes = []
for path in glob.glob(chemin):# parcourt tous les textes du chemin et les ajoute dans une liste pour traiter langue par langue et non pas texte par texte
    texte = lirefich(path)
    texte_nettoye = nettoyer_texte(texte)
    liste_textes.append(texte_nettoye)
print("Il y a ", len(liste_textes), "textes")
textes_string = " ".join(liste_textes)# transforme la liste des textes en string pour utiliser la librairie spacy
print(textes_string[:500])

doc = nlp(textes_string)# utilisation de la librairie spacy pour et transforme le format string en format doc
# liste_tokens = liste_textes_string.split()
liste_tokens_split = tokeniser_split(textes_string)# utilisation de la fonction split() pour récupérer les tokens des textes de la langue choisie
liste_tokens_spacy = tokeniser_spacy(doc)# utilisation de la fonction Tokeniser de spacy pour récupérer les tokens
liste_lemmes = lemmatiser(doc)# utilisation de la fonction Lemmatizer de spacy pour récupérer les lemmes
liste_entites = trouver_entites(doc)# utilisation de la fonction Entity Recognizer de spacy
liste_vocab_auto = recuperer_vocab_auto(doc, liste_langues_nltk[i])# on récupère le vocabulaire des textes de la langue choisie
print(liste_vocab_auto[:500])
liste_vocab_manuel = recuperer_vocab_manu(doc)# idem
print(liste_vocab_manuel[:500])

sauvegarder_json(liste_vocab_manuel, "liste_vocab_manuel", langue)# enregistrement au format JSON du vocabulaire
sauvegarder_json(liste_vocab_auto, "liste_vocab_auto_propre", langue)# idem

In [ ]:
#CODE POUR CLUSTERING

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 11 11:16:20 2022

@author: antonomaz
"""


import numpy as np
from sklearn.cluster import AffinityPropagation
from sklearn.metrics import DistanceMetric
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import json
import glob
import re
from collections import OrderedDict
import matplotlib.pyplot as plt


def sauvegarder_json(dico, nom_liste, langue):
    nom_fichier = f"{nom_liste}_{langue}.json"
    with open(nom_fichier, "w", encoding="utf-8") as json_file:
        json.dump(dico, json_file, ensure_ascii=False, indent=2)
    print(f"Cluster sauvegardé dans {nom_fichier} avec succès !")

def n_gramme(listee,nb=int(input("donner in chiffre"))):
    nbr_gram=[]
    for elt in listee:
        if len(elt) > 1:
            nbr_gram.append(elt[:nb])
    return nbr_gram

def nomfichier(chemin):
    nomfich= chemin.split("/")[-1]
    nomfich= nomfich.split(".")
    nomfich= ("_").join([nomfich[0],nomfich[1]])
    return nomfich

def lire_json(chemin_fichier_json):
    with open(chemin_fichier_json, "r", encoding="utf-8") as r:
        fich_json = json.load(r)
    return fich_json


chemin_entree = ["vocab_manuel_français.json","vocab_auto_français.json","vocab_manuel_anglais.json","vocab_auto_anglais.json"]


def cluster(chemin):
#for subcorpus in glob.glob(path_copora):
#    print("SUBCORPUS***",subcorpus)
 #   liste_nom_fichier =[]
    for path in glob.glob(chemin):
    #        print("PATH*****",path)
            liste_nom_fichier=[]
            nom_fichier = nomfichier(path)
    #        print(nom_fichier)
            liste= lire_json(path)
            liste_n_gramme= n_gramme(liste)
            print("les n_grammes ont été récupéré")

    #### FREQUENCE ########

            dic_mots={}
            i=0


            for mot in liste_n_gramme:

                if mot not in dic_mots:
                    dic_mots[mot] = 1
                else:
                    dic_mots[mot] += 1

            i += 1

            new_d = OrderedDict(sorted(dic_mots.items(), key=lambda t: t[0]))

            freq=len(dic_mots.keys())


            Set_00 = set(liste_n_gramme)
            Liste_00 = list(Set_00)
            dic_output = {}
            liste_words=[]
            matrice=[]

            for l in Liste_00:

                if len(l)!=1:
                    liste_words.append(l)


            try:
                words = np.asarray(liste_words)
                for w in words:
                    liste_vecteur=[]


                    for w2 in words:

                            V = CountVectorizer(ngram_range=(2,3), analyzer='char')
                            X = V.fit_transform([w,w2]).toarray()
                            distance_tab1=sklearn.metrics.pairwise.cosine_distances(X)
                            liste_vecteur.append(distance_tab1[0][1])

                    matrice.append(liste_vecteur)
                matrice_def=-1*np.array(matrice)


                affprop = AffinityPropagation(affinity="precomputed", damping= 0.6, random_state = None)

                affprop.fit(matrice_def)
                for cluster_id in np.unique(affprop.labels_):
                    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
                    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
                    cluster_str = ", ".join(cluster)
                    cluster_list = cluster_str.split(", ")

                    Id = "ID "+str(i)
                    for cle, dic in new_d.items():
                        if cle == exemplar:
                            dic_output[Id] ={}
                            dic_output[Id]["Centroïde"] = exemplar
                            dic_output[Id]["Freq. centroide"] = dic
                            dic_output[Id]["Termes"] = cluster_list

                    i=i+1
                return dic_output



            except :
               # print("**********Non OK***********", path)


                liste_nom_fichier.append(path)


               # continue

#ENREGISTRER LES N-GRAMMES EN FORMAT JSON

#bi_gramme_vocab_manuel_fr= sauvegarder_json(cluster(chemin_entree[0]), "bi_gramme_vocab_manuel", "fr")
#tri_gramme_vocab_manuel_fr= sauvegarder_json(cluster(chemin_entree[0]),"tri_gramme_vocab_manuel","fr")
#quatre_gramme= sauvegarder_json(cluster(chemin_entree[2]), "quatre_gramme_vocab_manuel", "ang")
#cinq_gramme= sauvegarder_json(cluster(chemin_entree[2]),"cinq_gramme_vocab_manuel","ang")

# #REPRESENTATION GRAPHIQUE DES CLUSTERS
#  #liste où on stockera les donnees pour le graphique
# cluster_len= []
# centroid= []

# entree_json= "les_n_grammes_json/cinq_gramme_vocab_manuel_ang.json"

# for path2 in glob.glob(entree_json):
#     fichier= lire_json(path2)

#     # calculer la longueure des clusters et les ajouter dans liste
#     for cluster2 in fichier:
#         cluster_length= len(fichier[cluster2]["Termes"])
#         cluster_len.append(cluster_length)
#         centroid.append(fichier[cluster2]["Centroïde"])

# #realisation des graphiques
# plt.figure(figsize=(20,14)) #taille du graphique
# for i , (length, centroids) in enumerate(zip(cluster_len,centroid)):
#     plt.scatter(i,length,s= length*15, alpha=0.7, label=f"Cluster{i}") #personnaliser les cercles du graphqiues
#     plt.annotate(centroids,(i,length),textcoords= "offset points", xytext=(0,6), ha="center",fontsize="8")# personaliser le titre des cercles

# plt.xlabel("cluster index")
# plt.ylabel("longueur des clusters")
# plt.title("taille des clusters pour cinq-grammes ang")
# plt.grid(True)
# #plt.savefig("graphique cinq-gramme ang")
# #plt.show()



In [ ]:
#CODE POUR LES REPRESENTATION GRAPHIQUE

#REPRESENTATION GRAPHIQUE

#token
width= 0.1 #specifie la largeur de chaque bar
token_langues=[22408,21306,18052] #donnee a representer dans notre graphique
langue=["fr","es","en"] #label qui corresponds à chaque donnee ci-dessus
x2= np.arange(len(langue)) # permet de representer le graphique correctement en associant les donnees avec le label qui le corresponds
fig2,ax2= plt.subplots() #doit etre redefini sinon les graph se superposent

graph2= plt.bar(range(len(token_langues)),token_langues) #dessine le graphique
plt.bar_label(graph2, padding=3) #permet d'ajouter des label au graphique
graph2[0].set_color("blue") #couleur de la premiere bar
graph2[1].set_color("pink") #couleur de la deuxième
graph2[2].set_color("purple") #couleur de la troisième

plt.title("nbr de tokens pour chaque langue")

ax2.set_xticks(x2+width, langue) # place correctement le label par rapport à la bar qui le corresponds

#plt.savefig("nbr tokens langues")
#plt.show()

#lemmatisation
lemmes_langues=[22408,21306,18052]
fig3, ax3= plt.subplots()

graph3= plt.bar(range(len(lemmes_langues)),lemmes_langues)
plt.bar_label(graph3, padding= 3)
graph3[0].set_color("red")
graph3[1].set_color("green")
graph3[2].set_color("orange")

plt.title("nbr de lemmes pour chaque langue")

ax3.set_xticks(x2+width, langue)

#plt.show()
#plt.savefig("nbr lemme langues")

#ent
ent_langues=[1062,1385,1500]
fig4, ax4= plt.subplots()

graph4= plt.bar(range(len(ent_langues)),ent_langues)
plt.bar_label(graph4, padding=3)
graph4[0].set_color("yellow")
graph4[1].set_color("cyan")
graph4[2].set_color("pink")

plt.title("nbr de ent pour chaque langues")

ax4.set_xticks(x2+width, langue)
#plt.show()
#plt.savefig("nbr ent langues")

#comparaison de la taille des vocab manuel et automatique
width= 0.1
vocab_langues=[12484,6895,17362,7033,15742,8744]
type_vocab=["en_nltk","en_manuel","es_nltk","es_manuel","fr_nltk","fr_manuel"]
x3= np.arange(len(type_vocab))
fig5,ax5= plt.subplots() #doit etre redefini sinon les graph se superposent

graph5= plt.bar(range(len(vocab_langues)),vocab_langues)
plt.bar_label(graph5, padding=3)
graph5[0].set_color("yellow")
graph5[1].set_color("yellow")
graph5[2].set_color("blue")
graph5[3].set_color("blue")
graph5[4].set_color("orange")
graph5[5].set_color("orange")

plt.title("comparaison des vocabulaires")

ax5.set_xticks(x3+width,type_vocab)

#plt.savefig("comparer vocab")
#plt.show()

In [ ]:
# GRAPHIQUE CLUSTER

# -*- coding: utf-8 -*-
"""
Created on Fri Mar  8 19:20:48 2024

@author: julco
"""
"""

"""
import matplotlib.pyplot as plt
import glob
import json

def lire_json(chemin):
    with open(chemin, "r", encoding="UTF-8") as f:
        chaine = json.load(f)
    return chaine

# Initialise des listes vides pour les remplir par la suite afin d'y stocker des informations pour le graphique
cluster_lengths = []
centroids = []

entree = ""
# Parcourt le dossier pour lire les fichiers JSON
for chemin in glob.glob(entree):
    nom_fichier = chemin.split("\\")[-1]
    print(nom_fichier)
    fichier = lire_json(chemin)

    # Permet de calculer la longueur des clusters et de les ajouter à une liste
    for cluster in fichier:
        cluster_length = len(fichier[cluster]["Termes"])
        cluster_lengths.append(cluster_length)
        centroids.append(fichier[cluster]["Centroïde"])

# Création d'un graphique qui représente la longueur des différents clusters sous forme de cercles plus ou moins important
plt.figure(figsize=(20, 14))# gère la taille du graphique
for i, (length, centroid) in enumerate(zip(cluster_lengths, centroids)):
    plt.scatter(i, length, s = length * 10, alpha =  0.7, label = f"Cluster {i}")# personnalisation des cercles du graphique
    plt.annotate(centroid, (i, length), textcoords = "offset points", xytext = (0,6), ha = "center", fontsize = "8")# permet de personnaliser le titre des cercles

# Personnalisation du graphique et enregsitrement de ce-dernier
plt.xlabel("Cluster Index")
plt.ylabel("Longueur des cluster")
plt.title("Longueur de cluster selon les centroides")
plt.grid(True)
# plt.savefig("Graphique_cluster.png")
# plt.legend()
plt.show()

